# Study based on Chapter 13

### 13.1. Tutorial Overview
In this section, we will develop a framework for grid searching SARIMA model hyperparameters for a given univariate time series forecasting problem. For more information on SARIMA for time series forecasting, see Chapter 5. We will use the implementation of SARIMA provided by the Statsmodels library. This model has hyperparameters that control the nature of the model performed for the series, trend and seasonality, specifically:
* order: A tuple p, d, and q parameters for the modeling of the trend.
* seasonal order: A tuple of P, D, Q, and m parameters for the modeling the seasonality
* trend: A parameter for controlling a model of the deterministic trend as one of ‘n’, ‘c’, ‘t’, and ‘ct’ for no trend, constant, linear, and constant with linear trend, respectively

### 13.2 Develop a Grid Search Framework

In [1]:
# grid search sarima hyperparameters
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import pandas as pd

###  13.1: Example of a function for making a SARIMA forecast

In [2]:
# one-step sarima forecast
def sarima_forecast(history, order=None, sorder=None, trend=None):
    # define model
    model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend,
    enforce_stationarity=False, enforce_invertibility=False)

    # fit model
    model_fit = model.fit(disp=False)

    # make one step forecast
    yhat = model_fit.predict(len(history), len(history))
    return yhat[0]

In [3]:
data = [10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]
print(data)
# data split
n_test = 4

[10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]


### 11.3. Accessory functions

In [4]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [5]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    predictions = list()
    
    # split dataset
    train, test = train_test_split(data, n_test)
    
    # seed history with training dataset
    history = [x for x in train]
    
    # step over each time-step in the test set
    for i in range(len(test)):        
        # fit model and make forecast for history
        yhat = sarima_forecast(history, cfg[0], cfg[1], cfg[2])
        
        # store forecast in list of predictions
        predictions.append(yhat)
        
        # add actual observation to history for the next loop
        history.append(test[i])
    
    # estimate prediction error
    error = measure_rmse(test, predictions)
    return error

In [12]:
data = [10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]
print(data)

# data split
n_test = 4


[10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0]


In [21]:
predictions = list()
    
# split dataset
train, test = train_test_split(data, n_test)
    
# seed history with training dataset
history = [x for x in train]

In [15]:
train, test

([10.0, 20.0, 30.0, 40.0, 50.0, 60.0], [70.0, 80.0, 90.0, 100.0])

In [22]:
history

[10.0, 20.0, 30.0, 40.0, 50.0, 60.0]

In [26]:
cfg = [(1, 1, 0), (0, 0, 0, 0), 'c']

In [28]:
for i in range(len(test)):        
    # fit model and make forecast for history
    yhat = sarima_forecast(history, cfg[0], cfg[1], cfg[2])
    print(history, yhat)

    # store forecast in list of predictions
    predictions.append(yhat)

    # add actual observation to history for the next loop
    history.append(test[i])

[10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0, 70.0, 80.0, 90.0, 100.0] 106.36363636363636
[10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0, 70.0, 80.0, 90.0, 100.0, 70.0] 80.0
[10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0, 70.0, 80.0, 90.0, 100.0, 70.0, 80.0] 83.33333333333333
[10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0, 70.0, 80.0, 90.0, 100.0, 70.0, 80.0, 90.0] 93.84615384615384


In [6]:
# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
    models = list()
    
    # define config lists
    p_params = [0, 1, 2]
    d_params = [0, 1]
    q_params = [0, 1, 2]
    t_params = ['n', 'c', 't', 'ct']
    P_params = [0, 1, 2]
    D_params = [0, 1]
    Q_params = [0, 1, 2]
    m_params = seasonal
    
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    models.append([(p,d,q), (P,D,Q,m), t])
    
    return models

                                

In [11]:
config_list[0]

[(0, 0, 0), (0, 0, 0, 0), 'n']

In [10]:
config_list = sarima_configs()
len(config_list)

1296

In [8]:
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
    if result is not None:
        print(' > Model[%s] %.3f' % (key, result))
    return (key, result)

In [9]:
# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
    scores = None
    if parallel:
        # execute configs in parallel
        executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
        tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
        scores = executor(tasks)
    else:
        scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
    
    # remove empty results
    scores = [r for r in scores if r[1] != None]
    
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    
    return scores

### 13.4 Example of demonstrating the grid search infrastructure.

